# Baseline SCA Measurements using Shunt Resistor on Target

The following script was used to capture the baseline measurements, first for T-Test, and then for CPA.

The firmware is also built in this notebook to keep a constant binary image for later comparison.

Note both sync & async captures leave the clock output enabled. This could be turned off as it increases noise a bit, but to leave the closest comparison between the two I've used the same binaries.

In [ ]:
import numpy as np
import chipwhisperer as cw
import os

In [ ]:
%run "functions.ipynb"

In [ ]:
data_dir = "d:/data_store"

## ChipWhisperer Configuration (Shared)

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' # 'TINYAES128C' or 'MBEDTLS'
SS_VER='SS_VER_1_1'

In [ ]:
scope = cw.scope(bitstream = r"C:\dev\cwhusky-fpga-jtag\fpga\vivado\cwhusky.runs\impl_no_ilas\cwhusky_top.bit")
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()

In [ ]:
scope.clock.clkgen_freq = 16E6
target.baud = 38400
scope.clock.adc_mul = 2
scope.adc.samples = 20000
scope.gain.mode = "low"
scope.gain.gain = 45

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
 for tap in tap_cmds:
    jtag_clkout(False)
    bypass_tap(tap["ID"], tap["CMD"])
    jtag_clkout(True)
    print("testing " + tap["Name"])
    group1, group2 = capture_ttest(10000, picoscope=False, splot=splot)
    
    #first = []
    #for g in group1:
    #    first.append(g[0])
    #avg = np.mean(first)
    #group1resync = []
    #for g in group1:
    #    if g[0] > avg:
    #        group1resync.append(g[:-2])
    #    else:
    #        group1resync.append(g[2:])
    # 
    #group2resync = []
    #for g in group2:
    #    if g[0] > avg:
    #        group2resync.append(g[:-2])
    #    else:
    #        group2resync.append(g[2:])
    
    nptsave("jtag_16mhztdo_32msps_noresync_{:s}_10k".format(tap["Name"]), group1, group2)    

In [ ]:
scope.clock

In [ ]:
 group1, group2 = capture_ttest(1000, picoscope=False, splot=splot)

In [ ]:
from scipy.stats import ttest_ind
import matplotlib.pylab as plt
from scipy import signal
t_val = ttest_ind(group1, group2, axis=0, equal_var=False)[0]

In [ ]:
import matplotlib.pylab as plt
plt.figure()
plt.plot(t_val)

In [ ]:
%matplotlib notebook
plt.plot(np.mean(group1, axis=0))

In [ ]:
def jtag_clkout(enabled):
    if enabled:
        data = 0x08
    else:
        data = 0x00

    CODE_READ = 0x80
    CODE_WRITE = 0xC0
    scope.userio.oa.sendMessage(CODE_WRITE, "USERIO_DEBUG_DRIVEN", [data])
    
    # Can use this to check write worked
    #scope.userio.oa.sendMessage(CODE_READ, "USERIO_DEBUG_DRIVEN")
    

In [ ]:
def read_tdo_status():
    pins = scope.userio.status
    if pins & (1<<3):
        return True
    else:
        return False
    
def write(tms, tdi):
    old = scope.userio.drive_data
    old &= ~(1<<6 | 1<<7)
    if tms:
        old |= 1<<6
    if tdi:
        old |= 1<<7
    
    scope.userio.drive_data = old
    scope.userio.drive_data = old | (1<<5)
    scope.userio.drive_data = old & ~(1<<5)

In [ ]:
def setup_bypass(verbose=True):
    #Take control of TDI, TMS, TCK
    scope.userio.direction = 0b11100000
    
    write(1, 1)
    write(1, 1)
    write(1, 1)
    write(1, 1)
    write(1, 1)

    write(0, 1) #
    write(1, 1)
    write(1, 1)
    write(0, 1)
    write(0, 1)

    #Send a bunch of 1's to force bypass mode
    for i in range(0, 10):
        write(0, 1)

    #exit shift-IR state
    write(1,1)

    write(1, 1)
    write(1, 1)
    write(0, 1)
    write(0, 1)

    for i in range(0, 10):
        write(0, 0)

    tdo_result = []

    for i in range(0, 10):
        tdo_result.append(read_tdo_status())
        if i == 0:
            write(0, 1)
        else:
            write(0, 0)

    if tdo_result[0:10] == [False, True, False, False, False, False, False, False, False, False]:
        if verbose:
            print("JTAG Setup successful - bypass mode enabled, saw '1' sequence successfuly")
        return True
    else:
        if verbose:
            print("JTAG Setup not successful")
            print(tdo_result)
        return False

In [ ]:
jtag_clkout(False)
setup_bypass()

In [ ]:
jtag_clkout(False)

In [ ]:
scope.clock

In [ ]:
| ACCESS_AUX_TAP_NPC | 10000     | Enables access to the NPC TAP controller                                         |
| ACCESS_AUX_TAP_OnCE| 10001     | Enables access to the primary e200 OnCE TAP controller (Primary CPU, core 0)      |
| ACCESS_AUX_TAP_eTPU| 10010     | Enables access to the eTPU Nexus TAP controller (eTPU_A, eTPU_B, CDC_AB)          |
| ACCESS_AUX_TAP_NXDM| 10011     | Enables access to the eDMA_A Nexus TAP controller (for Data Trace)                 |
| ACCESS_AUX_TAP_NXFR| 10100     | Enables access to the FlexRay Nexus TAP controller (for Data Trace)               |
| ACCESS_AUX_TAP_eTPU_SECONDARY | 10110 | Enables access to a secondary set of eTPU modules (eTPU_C, eTPU_D, CDC_CD) |
| ACCESS_AUX_TAP_NXDM_B      | 10111     | Enables access to the eDMA_B Nexus TAP controller (for Data Trace)  |
| ACCESS_AUX_TAP_OnCE1       | 11001     | Enables access to the secondary OnCE TAP controller (core 1)        |

In [ ]:
from jtagbs import JTAGBS, JTAGCWUserIO
interface = JTAGCWUserIO(scope)
jtag = JTAGBS(interface)

#jtag.init_scanchain()
interface.scan_init_chain(True)

print(jtag.list_devids())
print(jtag.list_devices())

In [ ]:
def bypass_tap(tap=None, bypass_cmd = 0b11111):
    interface.read_DR(0, pause_dr=True)
    if tap:
        interface.write_IR(tap, 5, True)
    interface.write_IR(bypass_cmd, bin(bypass_cmd).count('1'), True)
    return check_bypass()

In [ ]:
def check_bypass(verbose=True):
    
    #write(1, 1)
    write(1, 1)
    write(0, 1)
    write(0, 1)

    for i in range(0, 10):
        write(0, 0)

    tdo_result = []

    for i in range(0, 10):
        tdo_result.append(read_tdo_status())
        if i == 0:
            write(0, 1)
        else:
            write(0, 0)
    if tdo_result[0:10] == [False, True, False, False, False, False, False, False, False, False]:
        if verbose:
            print("JTAG Setup successful - bypass mode enabled, saw '1' sequence successfuly")
        return True
    else:
        if verbose:
            print("JTAG Setup not successful")
            print(tdo_result)
        return False

In [ ]:
bypass_tap(None,   0b11111) #Main TAP
bypass_tap(0b10000, 0b1111) #TAP_NPC, 4-bit
bypass_tap(0b10001, 0b1111111111) #TAP_ONCE, 10-bit
bypass_tap(0b10010, 0b1111) #TAP_eTPU, 4-bit
bypass_tap(0b10011, 0b1111) #TAP_NXDM, 4-bit
bypass_tap(0b10100, 0b1111) #TAP_NXFR, 4-bit
bypass_tap(0b10110, 0b1111) #TAP_eTPU2, 4-bit
bypass_tap(0b10111, 0b1111) #TAP_NXDM_B, 4-bit
bypass_tap(0b11001, 0b1111111111) #TAP_ONCE_1, 10-bit

In [ ]:
tap_cmds = [{"ID":None,    "CMD":0b11111, "Name":"Main"},
            {"ID":0b10000, "CMD":0b1111, "Name":"NPC"},
            {"ID":0b10001, "CMD":0b1111111111, "Name":"ONCE"},
            {"ID":0b10010, "CMD":0b1111, "Name":"eTPU"},
            {"ID":0b10011, "CMD":0b1111, "Name":"NXDM"},
            {"ID":0b10100, "CMD":0b1111, "Name":"NXFR"},
            {"ID":0b10110, "CMD":0b1111, "Name":"eTPU2"},
            {"ID":0b10111, "CMD":0b1111, "Name":"NXDM_B"},
            {"ID":0b11001, "CMD":0b1111111111, "Name":"ONCE_1"},
            ]          

In [ ]:
jtag_clkout(True)

In [ ]:
import time

scope.LA.enabled = True
scope.LA.oversampling_factor = 2
scope.LA.downsample = 1
scope.LA.capture_group = 'USERIO 20-pin'
scope.LA.trigger_source = "capture"
scope.LA.capture_depth = 16376

In [ ]:
scope.LA.arm()

In [ ]:
scope.LA.trigger_now()

In [ ]:
raw = scope.LA.read_capture_data()
tck = scope.LA.extract(raw, 5)
tdo = scope.LA.extract(raw, 3)
tdi = scope.LA.extract(raw, 7)
tms = scope.LA.extract(raw, 6)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1800)

xrange = range(len(tck))
O1 = o.line(xrange, tck + 6, line_color='black')
O2 = o.line(xrange, tdo + 4, line_color='blue')
O3 = o.line(xrange, tdi + 2, line_color='red')

legend = Legend(items=[
    LegendItem(label='TCK', renderers=[O1]),
    LegendItem(label='TDO', renderers=[O2]),
    LegendItem(label='TDI', renderers=[O3]),
])
o.add_layout(legend)

In [ ]:
show(o)

In [ ]:
np.all(tdi == tdo)

In [ ]:
max_freq = []

for tap in tap_cmds:
    
    print("testing " + tap["Name"])
    max_item = {"Name":tap["Name"], "Max":0}
    
    fails = 0
    for i in range(10, 100):        
        jtag_clkout(False)
        scope.clock.clkgen_freq = i*1E6
        time.sleep(0.05)
        bypass_tap(tap["ID"], tap["CMD"])
        jtag_clkout(True)

        scope.LA.arm()
        scope.LA.trigger_now()
        time.sleep(0.05)
        raw = scope.LA.read_capture_data()
        tck = scope.LA.extract(raw, 5)
        tdo = scope.LA.extract(raw, 3)
        tdi = scope.LA.extract(raw, 7)
        tms = scope.LA.extract(raw, 6)
    
        if (np.all(tdi == tdo)) == False and (np.all(tdi[1:] == tdo[0:-1])) == False:
            print("Failed at {:d} MHz".format(i*4))
            print(tdo[0:10])
            print(tdi[0:10])
            fails += 1
        else:
            fails = 0
            max_item["Max"] = i*4
        
        if fails > 6:
            break
    
    max_freq.append(max_item)

In [ ]:
print(max_freq)
np.save("mpc_5676r_max_freq_nobiterrors.npy", max_freq)

In [ ]:
max_freq

In [ ]:
scope.clock.clkgen_freq = 16E6

In [ ]:
scope.adc.offset = 5000
scope.adc.samples = 10000 

In [ ]:
## For SCA setup!!
#scope.gain.mode = "high"
#scope.gain.gain = 35


## For JTAG mixer output setup!!
scope.gain.mode = "low"
scope.gain.gain = 50

In [ ]:
jtag_clkout(False)
bypass_tap(0b10011, 0b1111) #TAP_NXDM, 4-bit
jtag_clkout(True)

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 300000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    
    #while ps.isReady() == False:
    #    continue
    
    #wave = ps.getDataV('A')
    
    wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 200 == 0:
        splot.update(wave) # wave is the name for the data for our power trace
        
        
    if i % 50000 == 0:
        try:
            print("Attempting save at {:d}".format(i))
            save_ets(waves, textins, textouts, keys, "d:/data_store/mpc5676r_jtag_16mhz_32msps_5koffset_cpa_300k.ets", overwrite=False)
            print("phew")
        except:
            print("Hmm... save failed, skipped!")
            pass

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/mpc5676r_jtag_16mhz_32msps_5koffset_cpa_300k.ets", overwrite=False)

In [ ]:
#save_ets(waves, textins, textouts, keys, "d:/data_store/mpc5676r_jtag_etpu_16mhz_32msps_250k_cpa_10koffset.ets", overwrite=False)



#SCA Reference
#save_ets(waves, textins, textouts, keys, "d:/data_store/mpc5676r_sca_16mhz_32msps_5k_cpa_25k.ets", overwrite=False)
save_ets(waves, textins, textouts, keys, "d:/data_store/mpc5676r_sca_jtagclockon_16mhz_32msps_21k_cpa_25k.ets", overwrite=False)

In [ ]:
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic() # default - fixed key, random plaintext

textins = []
textouts = []
waves = []
keys = []

N = 10000
for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next() # new plaintext, same key
    #ps.runBlock()
    trace = cw.capture_trace(scope, target, text, key) # set key, send plaintext, receive ciphertext, capture power trace
    if not trace:
        continue
    
    #while ps.isReady() == False:
    #    continue
    
    #wave = ps.getDataV('A')
    
    wave = trace.wave
 
    waves.append(wave)
    textins.append(trace.textin)
    textouts.append(trace.textout)
    keys.append(trace.key)

    # Update our plot with a new trace
    if i % 50 == 0:
        splot.update(wave) # wave is the name for the data for our power trace

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/mpc5676r_shunt_64msps_save.ets", overwrite=False)